In [50]:
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
os.chdir('/content/drive/MyDrive/HW3')

In [52]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [54]:
def readFile():
  dataset = []
  time_stamp = []
  for j in range(50):  
    data_csv = pd.read_csv('./training_data/target{}.csv'.format(j))
    h = len(data_csv)  
    gen = []
    con = []
    for i in range(0,h):    
      gen.append(data_csv["generation"][i])
      con.append(data_csv['consumption'][i])
    dataset.append((gen, con))

  max = len(dataset[0][0])
  for i in range(1, 50):
    if max < len(dataset[i][0]):
      max = len(dataset[i][0])
  del_dataset = []
  for i in range(50):
    if len(dataset[i][0]) == max:
      del_dataset.append(dataset[i])
  
  dataset = np.array(del_dataset)
  gen = np.sum(dataset[:,0,:], axis=0)/len(dataset)
  con = np.sum(dataset[:,1,:], axis=0)/len(dataset)
  return gen, con


In [55]:
def create_dataset(dataset, look_back_day=4):
  dataset = dataset.tolist()
  dataX, dataY = [], []
  end = len(dataset)-((look_back_day+1)*24)
  for i in range(0, end, 24):
    a = dataset[i:(i + (look_back_day)*24)]
    dataX.append(a)
    dataY.append(dataset[(i + (look_back_day)*24):(i + (look_back_day + 1)*24)])
  
  return np.array(dataX, dtype='float32'), np.array(dataY, dtype='float32')

In [56]:
def splitData(data_X, data_Y, look_back_day=4):
  train_size = int(len(data_X) * 0.9)
  test_size = len(data_X) - train_size
  print('train size : ', train_size)
  print('test size : ', test_size)
  train_X = data_X[:train_size]
  train_Y = data_Y[:train_size]
  test_X = data_X[train_size:]
  test_Y = data_Y[train_size:]

  train_X = train_X.reshape(-1, 1, look_back_day*24)
  train_Y = train_Y.reshape(-1, 1, 24)
  test_X = test_X.reshape(-1, 1, look_back_day*24)
  test_Y = test_Y.reshape(-1, 1, 24)

  train_x = torch.from_numpy(train_X)
  train_y = torch.from_numpy(train_Y)
  test_x = torch.from_numpy(test_X)
  test_y = torch.from_numpy(test_Y)
  return (train_x, train_y), (test_x, test_y)


In [57]:
class GRUNet(nn.Module):
  def __init__(self, input_size):
    super(GRUNet, self).__init__()
    self.rnn = nn.GRU(
        input_size=input_size,
        hidden_size=64,
        num_layers=3,
        batch_first=True,
        bidirectional=True
    )

    self.out = nn.Sequential(
        nn.Linear(128, 24)
    )

  def forward(self, x):
    x, _ = self.rnn(x)
    s, b, h = x.shape
    x = x.view(s*b, h)
    x = self.out(x)
    x = x.view(s, b, -1)

    return x

In [58]:
def RMSELoss(yhat, y):
    return torch.sqrt(torch.mean((yhat-y)**2))

In [59]:
def train(net, criterion, optimizer, train_data, test_data, epochs):
    min_loss = 1.0
    train_loss = None
    test_loss = None

    train_x, train_y = train_data
    test_x, test_y = test_data
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=60)
    for e in range(epochs):
        net.train()
        var_x, var_y = Variable(train_x).to(
            device), Variable(train_y).to(device)
        out = net(var_x)
        optimizer.zero_grad()
        loss = criterion(out, var_y)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()

        net.eval()
        with torch.no_grad():
            var_test_x, var_test_y = Variable(test_x).to(
                device), Variable(test_y).to(device)
            test_out = net(var_test_x)
            loss = criterion(test_out, var_test_y)
            test_loss = loss.item()
        scheduler.step(test_loss)
        if test_loss < min_loss:
            torch.save(net.state_dict(), './generationV1.pt')
            print('Save model : loss :', test_loss)
            min_loss = test_loss
        if (e + 1) % 5 == 0:
            print('Epoch: {}, Train Loss: {:.5f}, Test Loss: {:.5f}'.format(
                e + 1, train_loss, test_loss))

In [ ]:
look_back = 7
net = GRUNet(look_back*24).to(device)
criterion = RMSELoss
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

gen, con = readFile()
data_X, data_Y = create_dataset(gen, look_back_day=look_back)
train_data, test_data = splitData(data_X, data_Y, look_back)

train(net, criterion, optimizer, train_data, test_data, 1000)

In [62]:
model = torch.load('./generationV1.pt').to(device)
model.eval()
data_X = data_X.reshape(-1, 1, 168)
data_X = torch.from_numpy(data_X)
var_data = Variable(data_X).to(device)
pred_test = net(var_data) # 测试集的预测结果
pred_test = pred_test.view(-1).cpu().data.numpy()


AttributeError: ignored

In [ ]:
plt.figure(figsize=(40,10))
plt.plot(pred_test[:24], 'r', label='prediction')
plt.plot(gen[:24], 'b', label='real')
plt.show()